In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import math
import gc
import copy

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMRegressor

In [2]:
ATOMIC_NUMBERS = {
    'H': 1,
    'C': 6,
    'N': 7,
    'O': 8,
    'F': 9
}

In [3]:
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', 120)
# pd.set_option('display.max_columns', 120)

In [4]:
# train_dtypes = {
#     'molecule_name': 'category',
#     'atom_index_0': 'int8',
#     'atom_index_1': 'int8',
#     'type': 'category',
#     'scalar_coupling_constant': 'float32'
# }
train_csv = pd.read_csv('train.csv', index_col='id')
train_csv['molecule_index'] = train_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
train_csv = train_csv[['molecule_index', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant']]

C:\Users\Maynard\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
train_csv.head()

,molecule_index,atom_index_0,atom_index_1,type,scalar_coupling_constant
id,,,,,
0,1,1,0,1JHC,84.8076
1,1,1,2,2JHH,-11.2570
2,1,1,3,2JHH,-11.2548
3,1,1,4,2JHH,-11.2543
4,1,2,0,1JHC,84.8074


In [6]:
print('Shape: ', train_csv.shape)
print('Total: ', train_csv.memory_usage().sum())
train_csv.memory_usage()

Shape:  (4658147, 5)
Total:  204958468


Index                       37265176
molecule_index              18632588
atom_index_0                37265176
atom_index_1                37265176
type                        37265176
scalar_coupling_constant    37265176
dtype: int64

In [7]:
submission_csv = pd.read_csv('sample_submission.csv', index_col='id')

In [8]:
test_csv = pd.read_csv('test.csv', index_col='id')
test_csv['molecule_index'] = test_csv['molecule_name'].str.replace('dsgdb9nsd_', '').astype('int32')
test_csv = test_csv[['molecule_index', 'atom_index_0', 'atom_index_1', 'type']]

In [9]:
test_csv.head()

,molecule_index,atom_index_0,atom_index_1,type
id,,,,
4658147,4,2,0,2JHC
4658148,4,2,1,1JHC
4658149,4,2,3,3JHH
4658150,4,3,0,1JHC
4658151,4,3,1,2JHC


In [10]:
# structures_dtypes = {
#     'molecule_name': 'category',
#     'atom_index': 'int8',
#     'atom': 'category',
#     'x': 'float32',
#     'y': 'float32',
#     'z': 'float32'
# }
structures_csv = pd.read_csv('structures.csv')
structures_csv['molecule_index'] = structures_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
structures_csv = structures_csv[['molecule_index', 'atom_index', 'atom', 'x', 'y', 'z']]

In [11]:
structures_csv['atom'] = structures_csv['atom'].replace(ATOMIC_NUMBERS).astype('int8')
structures_csv.head(10)

,molecule_index,atom_index,atom,x,y,z
0,1,0,6,-0.012698,1.085804,0.008001
1,1,1,1,0.002150,-0.006031,0.001976
2,1,2,1,1.011731,1.463751,0.000277
3,1,3,1,-0.540815,1.447527,-0.876644
4,1,4,1,-0.523814,1.437933,0.906397
5,2,0,7,-0.040426,1.024108,0.062564
6,2,1,1,0.017257,0.012545,-0.027377
7,2,2,1,0.915789,1.358745,-0.028758
8,2,3,1,-0.520278,1.343532,-0.775543
9,3,0,8,-0.034360,0.977540,0.007602


In [12]:
print('Shape: ', structures_csv.shape)
print('Total: ', structures_csv.memory_usage().sum())
structures_csv.memory_usage()

Shape:  (2358657, 6)
Total:  87270389


Index                   80
molecule_index     9434628
atom_index        18869256
atom               2358657
x                 18869256
y                 18869256
z                 18869256
dtype: int64

In [13]:
def build_type_dataframes(base, structures, coupling_type):
    base = base[base['type'] == coupling_type].drop('type', axis=1).copy()
    base = base.reset_index()
    base['id'] = base['id'].astype('int32')
    structures = structures[structures['molecule_index'].isin(base['molecule_index'])]
    return base, structures

In [14]:
def add_coordinates(base, structures, index):
    df = pd.merge(base, structures, how='inner',
                  left_on=['molecule_index', f'atom_index_{index}'],
                  right_on=['molecule_index', 'atom_index']).drop(['atom_index'], axis=1)
    df = df.rename(columns={
        'atom': f'atom_{index}',
        'x': f'x_{index}',
        'y': f'y_{index}',
        'z': f'z_{index}'
    })
    return df

In [15]:
def add_atoms(base, atoms):
    df = pd.merge(base, atoms, how='inner',
                  on=['molecule_index', 'atom_index_0', 'atom_index_1'])
    return df

In [16]:
def merge_all_atoms(base, structures):
    df = pd.merge(base, structures, how='left',
                  left_on=['molecule_index'],
                  right_on=['molecule_index'])
    df = df[(df.atom_index_0 != df.atom_index) & (df.atom_index_1 != df.atom_index)]
    return df

In [17]:
def add_center(df):
    df['x_c'] = ((df['x_1'] + df['x_0']) * np.float32(0.5))
    df['y_c'] = ((df['y_1'] + df['y_0']) * np.float32(0.5))
    df['z_c'] = ((df['z_1'] + df['z_0']) * np.float32(0.5))

def add_distance_to_center(df):
    df['d_c'] = ((
        (df['x_c'] - df['x'])**np.float32(2) +
        (df['y_c'] - df['y'])**np.float32(2) + 
        (df['z_c'] - df['z'])**np.float32(2)
    )**np.float32(0.5))

def add_distance_between(df, suffix1, suffix2):
    df[f'd_{suffix1}_{suffix2}'] = ((
        (df[f'x_{suffix1}'] - df[f'x_{suffix2}'])**np.float32(2) +
        (df[f'y_{suffix1}'] - df[f'y_{suffix2}'])**np.float32(2) + 
        (df[f'z_{suffix1}'] - df[f'z_{suffix2}'])**np.float32(2)
    )**np.float32(0.5))

In [18]:
def add_distances(df):
    n_atoms = 1 + max([int(c.split('_')[1]) for c in df.columns if c.startswith('x_')])
    
    for i in range(1, n_atoms):
        for vi in range(min(4, i)):
            add_distance_between(df, i, vi)

In [19]:
def add_n_atoms(base, structures):
    dfs = structures['molecule_index'].value_counts().rename('n_atoms').to_frame()
    return pd.merge(base, dfs, left_on='molecule_index', right_index=True)

In [20]:
def build_couple_dataframe(some_csv, structures_csv, coupling_type, n_atoms=10):
    base, structures = build_type_dataframes(some_csv, structures_csv, coupling_type)
    base = add_coordinates(base, structures, 0)
    base = add_coordinates(base, structures, 1)
    
    base = base.drop(['atom_0', 'atom_1'], axis=1)
    atoms = base.drop('id', axis=1).copy()
    if 'scalar_coupling_constant' in some_csv:
        atoms = atoms.drop(['scalar_coupling_constant'], axis=1)
        
    add_center(atoms)
    atoms = atoms.drop(['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1'], axis=1)

    atoms = merge_all_atoms(atoms, structures)
    
    add_distance_to_center(atoms)
    
    atoms = atoms.drop(['x_c', 'y_c', 'z_c', 'atom_index'], axis=1)
    atoms.sort_values(['molecule_index', 'atom_index_0', 'atom_index_1', 'd_c'], inplace=True)
    atom_groups = atoms.groupby(['molecule_index', 'atom_index_0', 'atom_index_1'])
    atoms['num'] = atom_groups.cumcount() + 2
    atoms = atoms.drop(['d_c'], axis=1)
    atoms = atoms[atoms['num'] < n_atoms]

    atoms = atoms.set_index(['molecule_index', 'atom_index_0', 'atom_index_1', 'num']).unstack()
    atoms.columns = [f'{col[0]}_{col[1]}' for col in atoms.columns]
    atoms = atoms.reset_index()
    
    # downcast back to int8
    for col in atoms.columns:
        if col.startswith('atom_'):
            atoms[col] = atoms[col].fillna(0).astype('int8')
            
    atoms['molecule_index'] = atoms['molecule_index'].astype('int32')
    
    full = add_atoms(base, atoms)
    add_distances(full)
    
    full.sort_values('id', inplace=True)
    
    return full

In [21]:
def take_n_atoms(df, n_atoms, four_start=4):
    labels = []
    for i in range(2, n_atoms):
        label = f'atom_{i}'
        labels.append(label)

    for i in range(n_atoms):
        num = min(i, 4) if i < four_start else 4
        for j in range(num):
            labels.append(f'd_{i}_{j}')
    if 'scalar_coupling_constant' in df:
        labels.append('scalar_coupling_constant')
    return df[labels]

In [22]:
%%time
full = build_couple_dataframe(train_csv, structures_csv, '1JHN', n_atoms=10)
print(full.shape)

(43363, 73)
Wall time: 3.66 s


In [23]:
full.columns

Index(['id', 'molecule_index', 'atom_index_0', 'atom_index_1',
       'scalar_coupling_constant', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1',
       'atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'atom_7', 'atom_8',
       'atom_9', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'y_2',
       'y_3', 'y_4', 'y_5', 'y_6', 'y_7', 'y_8', 'y_9', 'z_2', 'z_3', 'z_4',
       'z_5', 'z_6', 'z_7', 'z_8', 'z_9', 'd_1_0', 'd_2_0', 'd_2_1', 'd_3_0',
       'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1', 'd_4_2', 'd_4_3', 'd_5_0', 'd_5_1',
       'd_5_2', 'd_5_3', 'd_6_0', 'd_6_1', 'd_6_2', 'd_6_3', 'd_7_0', 'd_7_1',
       'd_7_2', 'd_7_3', 'd_8_0', 'd_8_1', 'd_8_2', 'd_8_3', 'd_9_0', 'd_9_1',
       'd_9_2', 'd_9_3'],
      dtype='object')

In [24]:
df = take_n_atoms(full, 7)
# LightGBM performs better with 0-s then with NaN-s
df = df.fillna(0)
df.columns

Index(['atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'd_1_0', 'd_2_0',
       'd_2_1', 'd_3_0', 'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1', 'd_4_2', 'd_4_3',
       'd_5_0', 'd_5_1', 'd_5_2', 'd_5_3', 'd_6_0', 'd_6_1', 'd_6_2', 'd_6_3',
       'scalar_coupling_constant'],
      dtype='object')

In [25]:
X_data = df.drop(['scalar_coupling_constant'], axis=1).values.astype('float32')
y_data = df['scalar_coupling_constant'].values.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((34690, 23), (8673, 23), (34690,), (8673,))

In [26]:
LGB_PARAMS = {
    'objective': 'regression',
    'metric': 'mae',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'learning_rate': 0.0663,
    'num_leaves': 52,
    'min_child_samples': 126,
    'max_depth': -1,
    'bagging_fraction': 0.4649,
    'feature_fraction' : 0.7281,
    'reg_alpha': 0.6075,
    'reg_lambda': 0.1705,
}

In [27]:
def build_x_y_data(some_csv, coupling_type, n_atoms):
    full = build_couple_dataframe(some_csv, structures_csv, coupling_type, n_atoms=n_atoms)
    
    df = take_n_atoms(full, n_atoms)
    df = df.fillna(0)
    print(df.columns)
    
    if 'scalar_coupling_constant' in df:
        X_data = df.drop(['scalar_coupling_constant'], axis=1).values.astype('float32')
        y_data = df['scalar_coupling_constant'].values.astype('float32')
    else:
        X_data = df.values.astype('float32')
        y_data = None
    
    return X_data, y_data

In [28]:
def train_and_predict_for_one_coupling_type(coupling_type, submission, n_atoms, n_folds=5, n_splits=5, random_state=128):
    print(f'*** Training Model for {coupling_type} ***')
    
    X_data, y_data = build_x_y_data(train_csv, coupling_type, n_atoms)
    X_test, _ = build_x_y_data(test_csv, coupling_type, n_atoms)
    y_pred = np.zeros(X_test.shape[0], dtype='float32')

    cv_score = 0
    
    if n_folds > n_splits:
        n_splits = n_folds
    
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for fold, (train_index, val_index) in enumerate(kfold.split(X_data, y_data)):
        if fold >= n_folds:
            break

        X_train, X_val = X_data[train_index], X_data[val_index]
        y_train, y_val = y_data[train_index], y_data[val_index]

        model = LGBMRegressor(**LGB_PARAMS, n_estimators=10000, n_jobs = -1)
        model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
            verbose=1000, early_stopping_rounds=100)

        y_val_pred = model.predict(X_val)
        val_score = np.log(mean_absolute_error(y_val, y_val_pred))
        print(f'{coupling_type} Fold {fold}, logMAE: {val_score}')
        
        cv_score += val_score / n_folds
        y_pred += model.predict(X_test) / n_folds
        
        
    submission.loc[test_csv['type'] == coupling_type, 'scalar_coupling_constant'] = y_pred
    return cv_score

In [29]:
%%time

model_params = {
    '1JHN': 7,
    '1JHC': 10,
    '2JHH': 9,
    '2JHN': 9,
    '2JHC': 9,
    '3JHH': 9,
    '3JHC': 10,
    '3JHN': 10
}
N_FOLDS = 5
submission = submission_csv.copy()

cv_scores = {}
for coupling_type in model_params.keys():
    cv_score = train_and_predict_for_one_coupling_type(
        coupling_type, submission, n_atoms=model_params[coupling_type], n_folds=N_FOLDS)
    cv_scores[coupling_type] = cv_score

*** Training Model for 1JHN ***
Index(['atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'd_1_0', 'd_2_0',
       'd_2_1', 'd_3_0', 'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1', 'd_4_2', 'd_4_3',
       'd_5_0', 'd_5_1', 'd_5_2', 'd_5_3', 'd_6_0', 'd_6_1', 'd_6_2', 'd_6_3',
       'scalar_coupling_constant'],
      dtype='object')
Index(['atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'd_1_0', 'd_2_0',
       'd_2_1', 'd_3_0', 'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1', 'd_4_2', 'd_4_3',
       'd_5_0', 'd_5_1', 'd_5_2', 'd_5_3', 'd_6_0', 'd_6_1', 'd_6_2', 'd_6_3'],
      dtype='object')
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.274537	valid_1's l1: 0.3981
[2000]	training's l1: 0.203166	valid_1's l1: 0.366153
[3000]	training's l1: 0.164294	valid_1's l1: 0.352873
[4000]	training's l1: 0.138197	valid_1's l1: 0.345547
[5000]	training's l1: 0.119315	valid_1's l1: 0.341057
[6000]	training's l1: 0.10488	valid_1's l1: 0.338168
[7000]	training's l1: 0.093461	valid_

[1000]	training's l1: 1.02023	valid_1's l1: 1.07988
[2000]	training's l1: 0.856797	valid_1's l1: 0.948735
[3000]	training's l1: 0.762771	valid_1's l1: 0.879985
[4000]	training's l1: 0.697084	valid_1's l1: 0.835657
[5000]	training's l1: 0.647144	valid_1's l1: 0.804412
[6000]	training's l1: 0.606324	valid_1's l1: 0.780245
[7000]	training's l1: 0.57194	valid_1's l1: 0.761125
[8000]	training's l1: 0.542623	valid_1's l1: 0.745248
[9000]	training's l1: 0.51644	valid_1's l1: 0.731801
[10000]	training's l1: 0.493265	valid_1's l1: 0.720689
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.493265	valid_1's l1: 0.720689
1JHC Fold 4, logMAE: -0.3275480640153759
*** Training Model for 2JHH ***
Index(['atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'atom_7', 'atom_8',
       'd_1_0', 'd_2_0', 'd_2_1', 'd_3_0', 'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1',
       'd_4_2', 'd_4_3', 'd_5_0', 'd_5_1', 'd_5_2', 'd_5_3', 'd_6_0', 'd_6_1',
       'd_6_2', 'd_6_3', 'd_7_0', 'd_7_1', 'd_7_2', 

2JHN Fold 2, logMAE: -2.1118204382462142
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.137261	valid_1's l1: 0.171533
[2000]	training's l1: 0.103567	valid_1's l1: 0.14916
[3000]	training's l1: 0.0855368	valid_1's l1: 0.13902
[4000]	training's l1: 0.0737797	valid_1's l1: 0.13317
[5000]	training's l1: 0.0651256	valid_1's l1: 0.129318
[6000]	training's l1: 0.0585173	valid_1's l1: 0.126577
[7000]	training's l1: 0.0531875	valid_1's l1: 0.124608
[8000]	training's l1: 0.0488235	valid_1's l1: 0.122939
[9000]	training's l1: 0.0451657	valid_1's l1: 0.121663
[10000]	training's l1: 0.0420866	valid_1's l1: 0.120623
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.0420866	valid_1's l1: 0.120623
2JHN Fold 3, logMAE: -2.1150853618765204
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.137943	valid_1's l1: 0.17158
[2000]	training's l1: 0.103865	valid_1's l1: 0.149818
[3000]	training's l1: 0.0857478	val

3JHH Fold 1, logMAE: -1.8136457898250358
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.245723	valid_1's l1: 0.262434
[2000]	training's l1: 0.199924	valid_1's l1: 0.224222
[3000]	training's l1: 0.175274	valid_1's l1: 0.205254
[4000]	training's l1: 0.158405	valid_1's l1: 0.193336
[5000]	training's l1: 0.145508	valid_1's l1: 0.184752
[6000]	training's l1: 0.135351	valid_1's l1: 0.178326
[7000]	training's l1: 0.126929	valid_1's l1: 0.173355
[8000]	training's l1: 0.119786	valid_1's l1: 0.169289
[9000]	training's l1: 0.113661	valid_1's l1: 0.165981
[10000]	training's l1: 0.108264	valid_1's l1: 0.16312
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.108264	valid_1's l1: 0.16312
3JHH Fold 2, logMAE: -1.8132695018468645
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.243336	valid_1's l1: 0.260745
[2000]	training's l1: 0.198303	valid_1's l1: 0.223509
[3000]	training's l1: 0.174042	valid_1's l

[8000]	training's l1: 0.044939	valid_1's l1: 0.103344
[9000]	training's l1: 0.0415647	valid_1's l1: 0.102056
[10000]	training's l1: 0.0387322	valid_1's l1: 0.101027
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.0387322	valid_1's l1: 0.101027
3JHN Fold 0, logMAE: -2.2923662840332
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.122938	valid_1's l1: 0.144968
[2000]	training's l1: 0.0933467	valid_1's l1: 0.125307
[3000]	training's l1: 0.077445	valid_1's l1: 0.11632
[4000]	training's l1: 0.0670729	valid_1's l1: 0.111056
[5000]	training's l1: 0.0593109	valid_1's l1: 0.107361
[6000]	training's l1: 0.0534323	valid_1's l1: 0.104781
[7000]	training's l1: 0.0486716	valid_1's l1: 0.102866
[8000]	training's l1: 0.0447176	valid_1's l1: 0.101308
[9000]	training's l1: 0.041425	valid_1's l1: 0.100052
[10000]	training's l1: 0.0385746	valid_1's l1: 0.0990546
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.0385746	v

In [30]:
pd.DataFrame({'type': list(cv_scores.keys()), 'cv_score': list(cv_scores.values())})

,type,cv_score
0,1JHN,-1.104798
1,1JHC,-0.325175
2,2JHH,-1.870742
3,2JHN,-2.104783
4,2JHC,-1.253786
5,3JHH,-1.815818
6,3JHC,-1.149720
7,3JHN,-2.297123


In [31]:
np.mean(list(cv_scores.values()))

-1.490242925007926

In [32]:
submission[submission['scalar_coupling_constant'] == 0].shape

(0, 1)

In [34]:
submission.to_csv('lgb_bayes_10k.csv')